In [ ]:
from tensorflow.keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = train_X.reshape(-1, 28, 28, 1) / 255.0 
test_X = test_X.reshape(-1, 28, 28, 1) / 255.0 

print("train_X shape:", train_X.shape)
print("test_X shape:", test_X.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), 
                 input_shape=(28,28,1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D()) # pool_size=(2,2)는 기본값 
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filename = "model-digits.h5" 
checkpoint = ModelCheckpoint(filepath=filename,
                             monitor="val_accuracy", 
                             save_best_only=True, verbose=1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy",  
                               patience=5, verbose=1)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
model.fit(train_X, train_y, validation_data=(test_X, test_y),
          callbacks=[checkpoint, early_stopping], 
          batch_size=128, epochs=10)

In [ ]:
model.evaluate(test_X, test_y)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("mnist_keras_model.h5")

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if not ret: break # 반환되는 프레임이 없으면 종료
            
        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(bin_img,
                                               cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            (x,y), radius = cv2.minEnclosingCircle(contour)
            if radius < 5 : continue
            xs, xe = int(x-radius), int(x+radius)
            ys, ye = int(y-radius), int(y+radius)
            cv2.rectangle(bin_img, (xs,ys), (xe,ye), (200,0,0), 1)
            roi = bin_img[ys:ye, xs:xe]
            if roi.shape[0] * roi.shape[1] == 0:
                continue # ROI 배열의 행 또는 열의 크기가 0이면 resize시 예외 발생
            roi = cv2.resize(roi, dsize=(24,24))
            A = np.zeros((28,28))
            A[2:-2,2:-2] = roi
            A = A.reshape(-1,28,28,1)
            num = np.argmax(model.predict(A, verbose=0))
            cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200,0,0))
        cv2.imshow("Camera", bin_img)
        if cv2.waitKey(1) == 27: break # ESC
else :
    print("Camera not opened")
    
cap.release()
cv2.destroyAllWindows()